#Import libraries

In [ ]:
import os
import sys
from getpass import getpass
import json
import git
import dropbox
import torch.optim as optim

#Configure environment

In [ ]:
os.environ["ENVIRONMENT"] = "windows"
os.environ["REPO_DIR"] = git.Repo(search_parent_directories=True).git.rev_parse("--show-toplevel")
os.environ["ROOT_DIR"] = os.path.abspath(os.path.join(os.getenv("REPO_DIR"), os.pardir))
# Insert MODEL_ID here
os.environ["MODEL_ID"] = ""
os.environ["MODEL_DIR"] = f"{os.getenv('REPO_DIR')}/models/model_{os.getenv('MODEL_ID')}"

#Import repository resources

In [ ]:
sys.path.append(f"{os.getenv('REPO_DIR')}/src")
from helpers import *

sys.path.append(f"{os.getenv('MODEL_DIR')}/src")
from methods import *
from architecture import *

#Download datasets

In [ ]:
# dbx_access_token = getpass("Enter your DropBox access token: ")

In [ ]:
# download_datasets_from_dropbox(
#     dbx_access_token = dbx_access_token,
#     include_all_datasets = True,
#     use_thread = True
# )

#Work

In [5]:
num_epochs = 50
batch_size = 32
val_size = 20

In [ ]:
device = set_device()
model = initialize_model(device=device)
train_dataset, val_dataset, benchmark_dataset = create_datasets(
    device=device, include_all_datasets=True)
train_dataloader, val_dataloader, benchmark_dataloader = create_dataloaders(
    train_dataset=train_dataset, val_dataset=val_dataset,
    benchmark_dataset=benchmark_dataset, batch_size=batch_size)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [ ]:
model, loss_hist, best_val_performance = training_loop(
    model=model, criterion=criterion, optimizer=optimizer, train_dataloader=train_dataloader,
    val_dataloader=val_dataloader, device=device, num_epochs=num_epochs, auto_stop=True)

In [ ]:
visualize_loss(loss_hist)
visualize_results(model, dataset=train_dataset, device=device, output_threshold=.4)
